In [2]:
!pip install googletrans

  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15739 sha256=06171282038af44382cbbbde6b07f05c4383095eb3f2cf29958b34befcb17aaf
  Stored in directory: c:\users\big data\appdata\local\pip\cache\wheels\20\da\eb\a54579056f265eede0417df537dd56d3df5b9eb2b25df0003d
Successfully built googletrans


In [49]:
import time
import re
import json
import random
import googletrans
from pprint import pprint
translator = googletrans.Translator()

In [ ]:
def mongo_connect_build():
    global mycol
    client = pymongo.MongoClient(host="192.168.1.107", port=27017)
    db = client['test11111111111111111']
    mycol = db["original-recipe"]

In [ ]:
def load_data():
    mongo_connect_build()
    all = mycol.find()
    test = list(all)

    with open("test.json",'w')as f:
        try:
            js = json.dumps(test,default=json_util.default)
        except Exception as e:
            print(e)
            pass
        finally:
            f.write(js)

In [4]:
def symbol_clear(data):
    pat = re.compile(r"[a-zA-Z]+|[\u4e00-\u9fa5]+")
    clear_data = re.findall(pat,data)
    return clear_data

In [5]:
def eng_translate(data):
    eng_id = [];
    if data:
        for idx,para_name in enumerate(data):
            if re.search(r"[a-z]+",para_name):
                eng = translator.translate(para_name,dest='zh-tw')
                data[idx] = eng.text
                eng_id.append(idx)
        for idx,item in enumerate(eng_id):
            if eng_id[idx] != 0:
                if eng_id[idx]-eng_id[idx-1] != 1: 
                    continue
                data[item-1]  =  data[item-1] + data[item]
        return data
    else: 
        return '' 

In [6]:
def word_clear(data,special_word):
    if data:
        for idx,para in enumerate(data):
            for word in special_word:
                para = re.sub(word,'',para) 
                data[idx] = para
            data[idx] = re.sub('[a-zA-Z]+','',para)
            if len(data[idx])<=2 :    
                data[idx] = ''
        return ''.join(data)
    else: 
        return ''

In [17]:
with open('./foodBank/total_recipe_0924_ingredient_cleaned.json','r',encoding="utf-8") as f:
    total = json.loads(f.read())
with open('special_word.txt','r',encoding="utf-8") as f:
    special_word = f.read().split()

In [62]:
def main():
    global new_data
    new_data = []
    try:
        for idx, item in enumerate(total):
            data = symbol_clear(item['recipe'])
            data = eng_translate(data)
            data = word_clear(data,special_word)
            #data = word_clear(data,special_word_dict)
            if data != '': 
                item['recipe'] = data
                new_data.append(item)

            
            if idx % 5000 == 0:
                print(f'\r Data completed: {idx}',end='')
    except Exception as e: 
        pass
        print(e)
    finally:
        print('\n')
        print('lose ratio: {}'.format(round(len(new_data)/len(total),3)))
        # print(recipe)

In [63]:
if __name__ == '__main__':
    start_time = time.time()
    main()
    end_time = time.time()
    print('\nUpdating completed!')
    print(f'Time used: {end_time-start_time}')

 Data completed: 240000

lose ratio: 0.982

Updating completed!
Time used: 3368.238127708435


In [68]:
for i in new_data[:2]:
    print(i)

{'_id': {'$oid': '5f56036cfafca3ee40f639f1'}, 'recipe': '金黃玉子燒', 'tags': ['玉子焼', '玉子燒', '煎蛋捲'], 'url': 'https://icook.tw/recipes/99001', 'like': '3,800 ', 'time': 'unknown', 'image': 'https://imageproxy.icook.network/resize?height=600&nocrop=false&stripmeta=true&type=auto&url=http%3A%2F%2Ftokyo-kitchen.icook.tw.s3.amazonaws.com%2Fuploads%2Frecipe%2Fcover%2F99001%2F7894a541b7bcc225.jpg&width=800', 'quantity': 0, 'item': [['蛋', '2顆'], ['水', '50ml'], ['味霖', '1大匙'], ['鰹魚粉', '1小匙'], ['太白粉', '2小匙']], 'step': {'content': ['步驟1.將材料全部混合在一起.筷子用站立的方式混合約10來下就好了.，（沙拉油裡再準備個餐巾紙.用來抹在鍋子裡的）', '步驟2.蛋在混合時.用筷子把蛋白拉起夾斷.這個動作..等等再煎的時候.也比較不會一陀蛋白直接滑進鍋子裡.', '步驟3.用餐巾紙沾油之後.抹在鍋子裡後.開中小火.然後倒入薄薄的一層蛋液.然後看到蛋液像圖片中這種狀態時.就可以捲起來了.，（我總共約分3-4次.）', '步驟4.將蛋捲起來', '步驟5.全部捲完之後.在往外推..（只要記的.靠把手的是裡面.全部捲完後一定是往外推.這樣才會好捲.）', '步驟6.再倒入少許的蛋液.蛋液不要太厚..這樣才不會外面一下就焦黑了.右邊蛋的下面也要讓蛋液流進去才行喔..用筷子輕輕抬起來..讓蛋液也留過去.，一樣到圖片這樣的狀態時就可以把蛋捲起來了', '步驟7.再將蛋也是從外面往裡面（靠把手的地方）捲進來', '步驟8.捲完之後.還是一樣往外推.', '步驟9.直到把整個蛋液用完為止.', '步驟10.最後在放到竹簾上捲起來.兩端用橡皮筋捆起來就好.，等到放涼之後才會好切喔', '步驟

In [70]:
# write into new json
with open('./foodBank/recipe0925_V1.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(new_data))